<a href="https://colab.research.google.com/github/etuckerman/LLaMA-3.1-QA-Finetune/blob/main/LLaMA_3_1_QA_Finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) hf_KkxegmKUOMZKsifxFmGPXAeigeVzFFbZu

# import dataset (16 qna's)

In [2]:
import pandas as pd

# Define the data
data = {
    'Question': [
        "Does the Jayco Eagle have a queen size bed?",
        "What is the MSRP of the Jayco Eagle?",
        "How many people can sleep in the Jayco Eagle?",
        "Does the Jayco Eagle have a shower?",
        "Does the Keystone Cougar have a queen size bed?",
        "What is the MSRP of the Keystone Cougar?",
        "How many people can sleep in the Keystone Cougar?",
        "Does the Keystone Cougar have a shower?",
        "Does the Coachmen Clipper Sport have a queen size bed?",
        "What is the MSRP of the Coachmen Clipper Sport?",
        "How many people can sleep in the Coachmen Clipper Sport?",
        "Does the Coachmen Clipper Sport have a shower?",
        "Does the Fleetwood Fiesta have a queen size bed?",
        "What is the MSRP of the Fleetwood Fiesta?",
        "How many people can sleep in the Fleetwood Fiesta?",
        "Does the Fleetwood Fiesta have a shower?"
    ],
    'Answer': [
        "Yes, the Jayco Eagle has 1 queen size bed.",
        "The MSRP of the Jayco Eagle is $32,186.",
        "The Jayco Eagle can sleep up to 9 people.",
        "Yes, the Jayco Eagle comes with a standard shower.",
        "Yes, the Keystone Cougar has 1 queen size bed.",
        "The MSRP of the Keystone Cougar is $28,603.",
        "The Keystone Cougar can sleep up to 6 people.",
        "Yes, the Keystone Cougar comes with a standard shower.",
        "No, the Coachmen Clipper Sport does not have a queen size bed.",
        "The MSRP of the Coachmen Clipper Sport is $6,083.",
        "The Coachmen Clipper Sport can sleep up to 6 people.",
        "No, the Coachmen Clipper Sport does not have a shower available.",
        "No, the Fleetwood Fiesta does not have a queen size bed.",
        "The MSRP of the Fleetwood Fiesta is $7,231.",
        "The Fleetwood Fiesta can sleep up to 6 people.",
        "No, the Fleetwood Fiesta does not have a shower available."
    ]
}

In [3]:
import numpy as np

#save data as dataframe
df = pd.DataFrame(data)

#create a random split and assign labels
labels = np.random.choice(['train', 'test', 'validation'], size=len(df), p=[0.7, 0.15, 0.15])
df['split'] = labels

In [4]:
#save data as csv
df.to_csv('qa_examples.csv', index=False)


In [5]:
#pip dump
!pip install transformers datasets
!pip install scikit-learn
!pip install bitsandbytes


  Using cached bitsandbytes-0.43.3-py3-none-manylinux_2_24_x86_64.whl.metadata (3.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 16.2 MB/s eta 0:00:00


## Load dataset

In [6]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files='qa_examples.csv', split='train')

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
dataset

Dataset({
    features: ['Question', 'Answer', 'split'],
    num_rows: 16
})

# Fine-Tuning the LLaMA 3.1 Model

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [9]:
#model name
#model_id = "meta-llama/Meta-Llama-3.1-70B-Instruct"
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct" #temp model

In [10]:
#load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [11]:
#load model with low memory footprint (using model checkpoint)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [25]:
from bitsandbytes.optim import Adam8bit

#not enough cuda ram, use bitsandbytes to quantize
# optimizer for 4-bit precision
optimizer = Adam8bit(model.parameters(), lr=3e-5)


In [26]:
#check if  tokenizer has a padding token
if tokenizer.pad_token is None:
    #if not, set one
    #use end of sentence token as pad_token or add a new pad token
    tokenizer.pad_token = tokenizer.eos_token
    if tokenizer.pad_token is None:
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        tokenizer.pad_token = '[PAD]'

In [27]:
def preprocess_data(examples):
    inputs = examples['Question']
    targets = examples['Answer']
    model_inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=256)

    #tokenize labels (answers)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, padding="max_length", truncation=True, max_length=256)

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

In [28]:
#convert to Hugging Face Dataset
hf_dataset = dataset.from_pandas(df)

In [29]:
#tokenize the dataset using AutoTokenizer
tokenized_dataset = hf_dataset.map(preprocess_data, batched=True)

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [30]:
print(tokenized_dataset)

Dataset({
    features: ['Question', 'Answer', 'split', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 16
})


In [31]:
train_dataset = tokenized_dataset.filter(lambda example: example['split'] == 'train')
validation_dataset = tokenized_dataset.filter(lambda example: example['split'] == 'validation')


Filter:   0%|          | 0/16 [00:00<?, ? examples/s]

Filter:   0%|          | 0/16 [00:00<?, ? examples/s]

In [32]:
print(train_dataset)

Dataset({
    features: ['Question', 'Answer', 'split', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 11
})


In [33]:
print(validation_dataset)

Dataset({
    features: ['Question', 'Answer', 'split', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 3
})


In [34]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",  # where to save the model
    eval_strategy="epoch",  # evaluate at end of each epoch
    save_strategy="epoch",  # save model at end of each epoch
    save_steps=100,  # save checkpoint every 100 steps
    learning_rate=3e-5,  # small lr for fine-tuning
    per_device_train_batch_size=1,  # **Reduced batch size
    per_device_eval_batch_size=1,  # **Reduced batch size
    num_train_epochs=5,  # enough epochs for small dataset
    weight_decay=0.01,  # regularize to prevent overfitting
    logging_dir="./logs",  # where to save logs
    logging_steps=10,  # log every 10 steps
    save_total_limit=2,  # keep only 2 recent checkpoints
    load_best_model_at_end=True,  # use best model at end
    gradient_accumulation_steps=4, # accumulate gradients over 2 steps
    fp16=True,  # enable mixed precision training
)

In [35]:
from transformers import Trainer
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding( #auto pad input seq to same length within a batch
    tokenizer=tokenizer  # ensures padding is done based on the tokenizer's requirements
)

model.config.gradient_checkpointing = True # enable gradient checkpointing

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    optimizers=(optimizer, None)  #optimizer for 8-bit precision

)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


RuntimeError: You can't move a model that has some modules offloaded to cpu or disk.

In [ ]:

# start training
trainer.train()
